<a href="https://colab.research.google.com/github/AndreySukhanov/Deep_Learning/blob/main/cats/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import matplotlib.pyplot as plt
from google.colab import files

# Подгрузка датасета от Microsoft

In [49]:
train, _ = tfds.load('cats_vs_dogs', split=['train[:100%]'], with_info=True, as_supervised=True)

# Функция для изменения размеров изображения

In [50]:
SIZE = (224, 224)
def resize_image(img, label):
  img = tf.cast(img, tf.float32)
  img = tf.image.resize(img, SIZE)
  img /= 255.0
  return img, label

# Уменьшаем размеры всех изображений, полученных из датасета

In [51]:
train_resized = train[0].map(resize_image)
train_batches = train_resized.shuffle(1000).batch(16)

# Создание основного слоя для создания модели

In [52]:
base_layers = tf.keras.applications.MobileNetV2(input_shape = (SIZE[0], SIZE[1], 3), include_top = False)

# Создание модели нейронной сети

In [53]:
from keras.backend import binary_crossentropy
model = tf.keras.Sequential([
    base_layers,
    GlobalAveragePooling2D(),
    Dropout(0.2),
    Dense(1)
])
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics =['accuracy'])


# Обучение нейронной сети (наши картинки, одна итерация обучения)

In [ ]:
model.fit(train_batches, epochs=1)

  33/1454 [..............................] - ETA: 1:24:33 - loss: 0.3894 - accuracy: 0.8617

# Функция для подгрузки изображений

In [ ]:
files.upload()

# Название изображений

In [ ]:
images = ['1 (1).jpg', '1 (2).jpg', '1 (3).jpg', '1 (4).jpg', '1 (5).jpg', '1 (6).jpg', '1 (7).jpg', '1 (8).jpg', '1 (9).jpg', '1 (10).jpg', '1 (11).jpg']

# Перебор изображений

In [ ]:
for i in images:
	img = load_img(i)
	img_array = img_to_array(img)
	img_resized, _ = resize_image(img_array, _)
	img_expended = np.expand_dims(img_resized, axis=0)
	prediction = model.predict(img_expended)
	plt.figure()
	plt.imshow(img)
	label = 'Собачка' if prediction > 0 else 'Кошка'
	plt.title('{}'.format(label))